<a href="https://colab.research.google.com/github/lblogan14/data_structures_and_algorithms/blob/master/ch10_maps_ht_sl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#10.1 Maps and Dictionaries
Dictionaries in Python are commonly known as **associate arrays** or **maps**. The abstraction, *dictionary* or *map* uses unique *keys* to map to associated *values*. The keys are assumed to be unique, but the values are not necessarily unique. Unlike a standard array, indices for a map need not be consecutive nor even numeric.

##10.1.1 The Map ADT
A map $M$ supports the following behaviors:
* `M[k]`: Return the value `v` associated with key `k` in map `M`, if
one exists; otherwise raise a `KeyError`. In Python, this is
implemented with the special method `__getitem__` .
* `M[k] = v`: Associate value `v` with key `k` in map `M`, replacing the existing
value if the map already contains an item with key
equal to `k`. In Python, this is implemented with the special
method `__setitem__` .
* `del M[k]`: Remove from map `M` the item with key equal to `k`; if `M`
has no such item, then raise a `KeyError`. In Python, this is
implemented with the special method `__delitem__` .
* `len(M)`: Return the number of items in map `M`. In Python, this is
implemented with the special method `__len__` .
* `iter(M)`: The default iteration for a map generates a sequence of
keys in the map. In Python, this is implemented with the
special method `__iter__` , and it allows loops of the form,
for `k` in `M`.

Additional behaviors:
* `k in M`: Return `True` if the map contains an item with key `k`. In
Python, this is implemented with the special `__contains__`
method.
* `M.get(k, d=None)`: Return `M[k]` if key `k` exists in the map; otherwise return
default value `d`. This provides a form to query `M[k]` without
risk of a `KeyError`.
* `M.setdefault(k, d)`: If key `k` exists in the map, simply return `M[k]`; if key `k`
does not exist, set `M[k] = d` and return that value.
* `M.pop(k, d=None)`: Remove the item associated with key `k` from the map and
return its associated value `v`. If key `k` is not in the map,
return default value `d` (or raise `KeyError` if parameter `d` is
`None`).
* `M.popitem()`: Remove an arbitrary key-value pair from the map, and return
`a(k,v)` tuple representing the removed pair. If map is
empty, raise a `KeyError`.
* `M.clear()`: Remove all key-value pairs from the map.
* `M.keys()`: Return a set-like view of all keys of `M`.
* `M.values()`: Return a set-like view of all values of `M`.
* `M.items()`: Return a set-like view of `(k,v)` tuples for all entries of `M`.
* `M.update(M2)`: Assign `M[k] = v` for every `(k,v)` pair in map `M2`.
* `M == M2`: Return `True` if maps `M` and `M2` have identical key-value
associations.
* `M != M2`: Return `True` if maps `M` and `M2` do not have identical keyvalue
associations.

##10.1.2 Application: Counting Word Frequencies
####Counting word frequencies in a document

In [0]:
freq = {}
for piece in open(filename).read().lower().split():
  # only consider alphabetic characters within this piece
  word = ' '.join(c for c in piece if c.isalpha())
  if word: # require at least one alphabetic character
    freq[word] = 1 + freq.get(word, 0)
    
max_word = ' '
max_count = 0
for (w,c) in freq.items(): # (key, value) tuples represent (word, count)
  if c > max_count:
    max_word = w
    max_count = c
print('The most frequent word is', max_word)
print('Its number of occurrences is', max_count)

##10.1.3 Python's MutableMapping Abstract Base Class
The Python's `collections` module provides two abstract base classes: `Mapping` and `MutableMapping`. The `Mapping` class includes all nonmutating methods supported by Python's `dict` class. The `MutableMapping` class extends to include the mutating methods, such as `__getitem__`, `__setitem__`, `__delitem__`, `__len__` and `__iter__`.

##10.1.4 `MapBase` Class
The `MapBase` class is itself a subclass of the `MutableMapping` class and provides additional support for the composition design pattern.

In [0]:
class MapBase(MutableMapping):
  '''Our own abstract base class that includes a nonpublic _Item class'''
  
  #---------------------nested _Item class------------------------------
  class _Item:
    '''Lightweight composite to store key-value pairs as map items'''
    __slots__ = '_key', '_value'
    
    def __init__(self, k, v):
      self._key = k
      self._value = v
      
    def __eq__(self, other):
      return self._key == other._key # compare items based on their keys
    
    def __ne__(self, other):
      return not (self == other) # opposite of __eq__
    
    def __It__(self, other):
      return self._key < other._key # compare items based on their keys

##10.1.5 Simple Unsorted Map Implementation

In [0]:
class UnsortedTableMap(MapBase):
  '''Map implementation using an unsorted list'''
  
  def __init__(self):
    '''Create a empty map'''
    self._table = [] # list of _Item's
    
  def __getitem__(self, k):
    '''Return value associated with key k (raise KeyError if not found)'''
    for item in self._table:
      if k == item._key:
        return item._value
    raise KeyError('Key Error: ' + repr(k))
    
  def __setitem__(self, k, v):
    '''Assign value v to key k, overwriting existing value if present'''
    for item in self._table:
      if k == item._key: # found a match
        item._value = v # reassign value
        return # and quit
    # did not find match for key
    self._table.append(self._Item(k,v))
    
  def __delitem(self, k):
    '''Remove item associted with key k (raise KerError if not found)'''
    for j in range(len(self._table)):
      if k == self._table[j]._key: # found a match
        self._table.pop(j) # remove item
        return # and quit
    raise KeyError('Key Error: '+ repr(k))
    
  def __len__(self):
    '''Return number of items in the map'''
    return len(self._table)
  
  def __iter__(self):
    '''Generate iteration of the map's keys '''
    for item in self._table:
      yield item._key # yield the KEY

Each of these methods runs in $O(n)$ time on a map with $n$ items.

#10.2 Hash Tables
A map $M$ supports the abstraction of using keys as indices with a syntax such as `M[k]`. The novel concept for a hash table is the use of a **hash function** to map general keys to corresponding indices in a table. Ideally, keys will be well distributed in the
range from 0 to N−1 by a hash function, but in practice there may be two or more
distinct keys that get mapped to the same index. As a result, the table $M$ is conceptualized as a **bucket array**, in which each bucket may manage a collection of items that are sent to the specific index by the hash function. (To save space, an empty bucket may be replaced by `None`)

##10.2.1 Hash Functions
The goal of a **hash funciton**, $h$, is to map each key $k$ to an integer in the range $[0, N-1]$, where $N$ is the capacity of the bucket array for a hash table. Equipped with such a hash function, $h$, the main idea is to use the hash funtion value, $h(k)$, as an index into the bucket array, $A$, instead of the key $k$. That is, the item $(k, v)$ is stored in the bucket $A[h(k)]$.

If there are two or more keys with the same hash value, then two different items
will be mapped to the same bucket in $A$. This is so called a **collision** has occured. A hash function is "good" if it maps the keys in the map so as to sufficiently minimize collisions.

The evaluation of a hash function, $h(k)$, consists of two portions: a **hash code** that maps a key $k$ to an integer, and a **compression function** that maps the hash code to an integer within a range of indices, $[0, N-1]$, for a bucket array.

The reason behind using those two components is that the hash code portion of that computation is independent of a specific hash table size, which allows the development of a general hash code for each object that can be used for a hash table of any size; only the compression function depends upon the table size.

##Hash Codes
The hash code for $k$ is an integer which is computed and an arbitrary key $k$ is taken in the map when a hash function applies. This integer need not be in the range$[0, N-1]$. The set of hash codes assigned to the keys should avoid collisions as much as possible, because there is no hope for the compression function to avoid them if the hash codes of the keys cause collisions.

###Hash Codes in Python
`hash(x)` returns an integer value that serves as the hash cod for object `x`. Only *immutable* data types are deemed hashable in Python.

Instances of user-defined classes are treated as unhashable by default, with a
`TypeError` raised by the hash function. However, a function that computes hash
codes can be implemented in the form of a special method named `__hash__` within
a class. The returned hash code should reflect the immutable attributes of an instance. \\
For example, a `Color` class that maintains three numeric red, green, and blue components might implement the method as:

In [0]:
def __hash__(self):
  return hash( (self._red, self._green, self._blue) ) # hash combined tuple

An important rule to obey is that if a class defines equivalence through `__eq__` ,
then any implementation of hash must be consistent, in that if `x == y`, then
`hash(x) == hash(y)`.

##Compression Functions  
The compression function maps an integer hash code for a key object $k$ into the range $[0,N-1]$.

A good compression function minimizes the number of collisions for a given set of distinct hash codes

####Division method
maps an integer $i$ to $$i\mod N$$
where $N$, the size of the bucket array, is a fixed positive integer.

####MAD method
maps an integer $i$ to
$$[(ai+b)\mod p] \mod N$$
where $N$ is the size of the bucket array, $p$ is a prime number larger than $N$, and $a$ and $b$ are integers chosen at random from the interval $[0, p-1]$, with $a>0$. \\
This
compression function is chosen in order to eliminate repeated patterns in the set of
hash codes and get us closer to having a “good” hash function, that is, one such that
the probability any two different keys collide is $1/N$.

##10.2.2 Collision-Handling Schemes
The main idea of a hash table is to take a bucket array $A$, and a hash function, $h$, and use them to implement a map by storing each item $(k,v)$ in the bucket $A[h(k)]$. The collisions occur when two distinct keys, $k_1$ and $k_2$ have $h(k_1)=h(k_2)$.

###Separate Chaining
A simple and efficient way for dealing with collisions is to have each bucket $A[j]$ store its own secondary container, holding items $(k,v)$ such that $h(k)=j$. A natural choice for the secondary container is a small map instance implemented using a list, as described by the compression function above. This **collision resolution** rule is known as **separate chaining**.

###Open Addressing
The separate chaining rule has many nice properties, such as affording simple implementations
of map operations, but it nevertheless has one slight disadvantage:
It requires the use of an auxiliary data structure—a list—to hold items with colliding
keys. If space is at a premium (for example, if we are writing a program for a
small handheld device), then we can use the alternative approach of always storing
each item directly in a table slot. This approach saves space because no auxiliary
structures are employed, but it requires a bit more complexity to deal with collisions.
There are several variants of this approach, collectively referred to as **open
addressing** schemes.

####Linear Probing
A simple method for collision handling with open addressing is **linear probing**.
With this approach, if we try to insert an item $(k,v)$ into a bucket $A[ j]$ that is already
occupied, where $j = h(k)$, then we next try $A[( j+1) \mod N]$. If $A[( j+1) \mod N]$
is also occupied, then we try $A[( j+2) \mod N]$, and so on, until we find an empty
bucket that can accept the new item. Once this bucket is located, we simply insert
the item there.

##10.2.4 Python Hash Table Implementation

In [0]:
class HashMapBase(MapBase):
  '''Abstract base class for map using hash-table with MAD compression'''
  
  def __init__(self, cap=11, p=109345121):
    '''Create an empty hash-table map'''
    self._table = cap * [None]
    self._n = 0 # number of entries in the map
    self._prime = p # prime for MAD compression
    self._scale = 1 + randrange(p-1) # scale from 1 to p-1 for MAD
    self._shift = randrange(p) # shift from 0 to p-1 for MAD
    
  def _hash_function(self, k):
    return (hash(k)*self._scale + self._shift) % self._prime % len(self._table)
  
  def __len__(self):
    return self._n
  
  def __getitem__(self, k):
    j = self._hash_function(k)
    return self._bucket_getitem(j, k) # may raise KeyError
  
  def __setitem__(self, k, v):
    j = self._hash_function(k)
    self._bucket_setitem(j, k, v) # subroutine maintains self._n
    if self._n > len(self._table)//2: # keep load factor <= 0.5
      self._resize(2*len(self._table) - 1) # number 2^x - 1 is often prime
      
  def __delitem__(self, k):
    j = self._hash_function(k)
    self._bucket_delitem(j, k) # may raise KeyError
    self._n -= 1
    
  def _resize(self, c): # resize bucket array to capacity c
    old = list(self.items()) # use iteration to record existing items
    self._table = c * [None] # then reset table to desired capacity
    self._n = 0 # n recomputed during subsequent adds
    for (k,v) in old:
      self[k] = v # reinsert old key-value pair
    

* The bucket array is represented as a Python list, named `self._table`, with all
entries initialized to `None`.
* We maintain an instance variable `self._n` that represents the number of distinct
items that are currently stored in the hash table.
* If the load factor of the table increases beyond 0.5, we double the size of the
table and rehash all items into the new table.
* We define a `_hash_function` utility method that relies on Python’s built-in
hash function to produce hash codes for keys, and a randomized Multiply-
Add-and-Divide (MAD) formula for the compression function

The `HashMapBase` class has not implemented how a "bucket" should be represented. With **separate chaining**, each bucket will be an independent structure. With **open addressing**, there is no tangible container for each bucket; the "buckets" are effectively interleaved due to the probing sequences 

###Separate Chaining
####Concrete Implementation of Hash Table with Separate Chaining

In [0]:
class ChainHashMap(HashMapBase):
  '''Hash map implemented with separate chaining for collision resolution'''
  
  def _bucket_getitem(self, j, k):
    bucket = self._table[j]
    if bucket is None:
      raise KeyError('Key Error: ' + repr(k)) # no match found
    return bucket[k]
  
  def _bucket_setitem(self, j, k, v):
    if self._table[j] is None:
      self._table[j] = UnsortedTableMap() # bucket is new to the table
    oldsize = len(self._table[j])
    self._table[j][k] = v
    if len(self._table[j]) > oldsize: # key was new to the table
      self._n += 1 # increase overall map size
      
  def _bucket_delitem(self, j, k):
    bucket = self._table[j]
    if bucket is None:
      raise KeyError('Key Error: ' + repr(k)) # no match found
    del bucket[k] # may raise KeyError
    
  def __iter__(self):
    for bucket in self._table:
      if bucket is not None: # a nonempty slot
        for key in bucket:
          yield key

###Linear Probing with Open Addressing

In [0]:
class ProbeHashMap(HashMapBase):
  '''Hash map implemented with linear probing for collision resolution'''
  _AVAIL = object() # sentinal marks locations of previous deleteions
  
  def _is_available(self, j):
    '''Return True if index j is available in table'''
    return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL
  
  def _find_slot(self, j, k):
    '''Search for key k in bucket at index j
    
    Return (success, index) tuple:
    If match was found, success is True and index denotes its location.
    If no match was found, success is False and index denotes first available slot
    '''
    firstAvail = None
    while True:
      if self._is_available(j):
        if firstAvail is None:
          firstAvail = j # mark this as first avail
        if self._table[j] is None:
          return (False, firstAvail) # search has failed
      elif k == self._table[j]._key:
        return (True, j) # found a match
      j = (j + 1) % len(self._table) # keep looking (cyclically)
      
  def _bucket_getitem(self, j, k):
    found, s = self._find_slot(j, k)
    if not found:
      raise KeyError('Key Error: ' + repr(k)) # no match found
    return self._table[s]._value
  
  def _bucket_setitem(self, j, k, v):
    found, s = self._find_slot(j, k)
    if not found:
      self._table[s] = self.Item(k, v) # insert new item
      self._n += 1 # size has increased
    else:
      self._table[s]._value = v # overwrite existing
      
  def _bucket_delitem(self, j, k):
    found, s = self._find_slot(j, k)
    if not found:
      raise KeyError('Key Error: ' + repr(k)) # no match found
    self._table[s] = ProbeHashMap._AVAIL # mark as vacated
    
  def __iter__(self):
    for j in range(len(self._table)): # scan entire table
      if not self._is_available(j):
        yield self._table[j]._key

#10.3 Sorted Maps
The sorted map ADT supports the following methods:
* `M.find_min()`: Return the `(key,value)` pair with minimum key
(or `None`, if map is empty).
* `M.find_max()`: Return the `(key,value)` pair with maximum key
(or `None`, if map is empty).
* `M.find_lt(k)`: Return the `(key,value)` pair with the greatest key that
is strictly less than `k` (or `None`, if no such item exists).
* `M.find_le(k)`: Return the `(key,value)` pair with the greatest key that
is less than or equal to `k` (or `None`, if no such item
exists).
* `M.find_gt(k)`: Return the `(key,value)` pair with the least key that is
strictly greater than `k` (or `None`, if no such item exists).
* `M.find_ge(k)`: Return the `(key,value)` pair with the least key that is
greater than or equal to `k` (or `None`, if no such item).
* `M.find_range(start, stop)`: Iterate all `(key,value)` pairs with `start <= key < stop`.
If `start` is `None`, iteration begins with minimum key; if
`stop` is `None`, iteration concludes with maximum key.
* `iter(M)`: Iterate all keys of the map according to their natural
order, from smallest to largest.
* `reversed(M)`: Iterate all keys of the map in reverse order; in Python,
this is implemented with the `__reversed__` method.

##10.3.1 Sorted Search Tables
The sorted search table has a space requirement that is $O(n)$, assuming the array is grown and shrunk to keep its size proportional to the number of items in the map. The primary advantage of this representation is that it allows us to use the **binary search** for a variety of efficient operations.

###Binary Search and Inexact Searches
The important realization is that while performing a binary search, we can determine
the index at or near where a target might be found. During a successful
search, the standard implementation determines the precise index at which the target
is found. During an unsuccessful search, although the target is not found, the
algorithm will effectively determine a pair of indices designating elements of the
collection that are just less than or just greater than the missing target.

###Implementation

In [0]:
class SortedTableMap(MapBase):
  '''Map implementation using a sorted table'''
  
  #--------------------nonpublic behaviors------------------------------
  def _find_index(self, k, low, high):
    '''Return index of the leftmost item with key greater than or equal to k
    
    Return high+1 if no such item qualifies
    
    That is, j will be returned such that:
      all items of slice table[low:j] have key < k
      all items of slice table[j:high+1] have key >= k
    '''
    if high < low:
      return high + 1 # no element qualifies
    else:
      mid = (low + high) // 2
      if k == self._table[mid]._key:
        return mid # found exact match
      elif k < self._table[mid]._key: 
        return self._find_index(k, low, mid - 1) # note: may return mid
      else:
        return self._find_index(k, mid + 1, high) # answer if right of mid
      
  def __init__(self):
    '''Create an empty map'''
    self._table = []
    
  def __len__(self):
    '''Return number of items in the map'''
    return len(self._table)
  
  def __getitem__(self, k):
    '''Return value associated with key k (raise KeyError if not found)'''
    j = self._find_index(k, 0, len(self._table) - 1)
    if j == len(self._table) or self._table[j]._key != k:
      raise KeyError('Key Error: ' + repr(k))
    return self._table[j]._value
  
  def __setitem__(self, k, v):
    '''Assign value v to key k, overwriting existing value if present'''
    j = self._find_index(k, 0, len(self._table) - 1)
    if j < len(self._table) and self._table[j]._key == k:
      self._table[j]._value = v # reassign value
    else:
      self._table.insert(j, self._Item(k,v)) # adds new item
      
  def __delitem__(self, k):
    '''Remove item associated with key k (raise KeyError if not found)'''
    j = self._find_index(k, 0, len(self._table) - 1)
    if j == len(self._table) or self._table[j]._key != k:
      raise KeyError('Key Error: ' + repr(k))
    self._table.pop(j) # delete item
    
  def __iter__(self):
    '''Generate keys of the map ordered from minimum to maximum'''
    for item in self._table:
      yield item._key
      
  def __reversed__(self):
    '''Generate keys of the map ordered from maximum to minimum'''
    for item in reversed(self._table):
      yield item._key
      
  def find_min(self):
    '''Return (key,value) pair with minimum key (or None if empty)'''
    if len(self._table) > 0:
      return (self._table[0]._key, self._table[0]._value)
    else:
      return None
    
  def find_max(self):
    '''Return (key,value) pair with maximum key (or None if empty)'''
    if len(self._table) > 0:
      return (self._table[-1]._key, self._table[-1]._value)
    else:
      return None
  
  def find_ge(self, k):
    '''Return (key,value) pair with least key greater than or equal to k'''
    j = self._find_index(k, 0, len(self._table) - 1) # j's key >= k
    if j < len(self._table):
      return (self._table[j]._key, self._table[j]._value)
    else:
      return None
    
  def find_lt(self, k):
    '''Return (key,value) pair with greatest key strictly less than k'''
    j = self._find_index(k, 0, len(self._table) - 1) # j's key >= k
    if j > 0:
      return (self._table[j-1]._key, self._table[j-1]._value) # Note: use j-1
    else:
      return None
  
  def find gt(self, k):
    '''Return (key,value) pair with least key strictly greater than k'''
    j = self._find_index(k, 0, len(self._table) − 1) # j s key >= k
    if j < len(self._table) and self._table[j]._key == k:
      j += 1 # advanced past match
    if j < len(self._table):
      return (self._table[j]._key, self._table[j]._value)
    else:
      return None
    
  def find_range(self, start, stop):
    '''Iterate all (key,value) pairs such that start <= key < stop
    
    If start is None, iteration begins with minimum key of map.
    If stop is None, iteration continues through the maximum key of map.
    '''
    if start is None:
      j = 0
    else:
      j = self._find_index(start, 0, len(self._table) - 1) # find first result
    while j < len(self._table) and (stop is None or self._table[j]._key < stop):
      yield (self._table[j]._key, self._table[j]._value)
      j += 1

* `__len__` , `find_min`, and `find_max` methods run in $O(1)$ time
* Iterating the keys of the table in $O(n)$ time
* `__getitem__` , `find_lt`, `find_gt`, `find_le`, and `find_ge` run in $O(\log n)$ worst-case time for binary search

#10.5 Sets, Multisets, and Multimaps
A **set** is an unordered collection of elements, without duplicates, that typically supports efficient membership tests. In essence, elements of a set are like keys of a map, but without any auxiliary values.

A **multiset** (also known as a **bag**) is a set-like container that allows duplicates.

A **multimap** is similar to a traditional map, in that it associates values with keys; however, in a multimap the same key can be mapped to multiple values. For example, the index of a book maps a given term to one or more locations at which the term occurs elsewhere in the book.

##10.5.1 The Set ADT
Python’s `collections` module defines abstract base classes that essentially mirror
these built-in classes. The `collections.Set` matches the concrete `frozenset` class, while the `collections.MutableSet` matches the concrete `set` class.

The set $S$ ADT supports the following methods:
* `S.add(e)`: Add element `e` to the set. This has no effect if the set
already contains `e`.
* `S.discard(e)`: Remove element `e` from the set, if present. This has no
effect if the set does not contain `e`.
* `e in S`: Return `True` if the set contains element `e`. In Python, this
is implemented with the special `__contains__` method.
* `len(S)`: Return the number of elements in set `S`. In Python, this
is implemented with the special method `__len__` .
* `iter(S)`: Generate an iteration of all elements of the set. In Python,
this is implemented with the special method `__iter__` .

Additional operations:
* `S.remove(e)`: Remove element `e` from the set. If the set does not contain `e`,
raise a `KeyError`.
* `S.pop()`: Remove and return an arbitrary element from the set. If the
set is empty, raise a `KeyError`.
* `S.clear()`: Remove all elements from the set.

##10.5.3 Implementing Sets, Multisets, and Multimaps

###Sets
Although sets and maps have very different public interfaces, they are really quite
similar. A set is simply a map in which keys do not have associated values. Any
data structure used to implement a map can be modified to implement the set ADT
with similar performance guarantees. An efficient set implementation should
abandon the `_Item` composite in the `MapBase` class and instead store
set elements directly in a data structure.

###Multisets
The same element may occur several times in a multiset. All of the data structures
we have seen can be reimplemented to allow for duplicates to appear as separate
elements. However, another way to implement a multiset is by using a map in
which the map key is a (distinct) element of the multiset, and the associated value
is a count of the number of occurrences of that element within the multiset. 

Python’s standard `collections` module includes a definition for a class named
`Counter` that is in essence a multiset. Formally, the `Counter` class is a subclass of
`dict`, with the expectation that values are integers, and with additional functionality
like a `most_common(n)` method that returns a list of the n most common elements.
The standard `__iter__` reports each element only once (since those are formally the
keys of the dictionary). There is another method named `elements()` that iterates
through the multiset with each element being repeated according to its count.

###Multimaps
Although there is no multimap in Python’s standard libraries, a common implementation
approach is to use a standard map in which the value associated with a
key is itself a container class storing any number of associated values. \\
The implementation below uses the standard `dict` class as the map, and a list of values as a composite value in the dictionary.

In [0]:
class MultiMap:
  '''A multimap class built upon use of an underlying map for storage'''
  _MapType = dict # Map type; can be redefined by subclass
  
  def __init__(self):
    '''Create a new empty multimap instance'''
    self._map = self._MapType() # create map instance for storage
    self._n = 0
    
  def __iter__(self):
    '''Iterate through all (k,v) pairs in multimap'''
    for k,secondary in self._map.items():
      for v in secondary:
        yield (k,v)
        
  def add(self, k, v):
    '''Add pair (k,v) to multimap'''
    container = self._map.setdefault(k, []) # create empty list, if needed
    container.append(v)
    self._n += 1
    
  def pop(self, k):
    '''Remove and return arbitrary (k,v) with key k (or raise KeyError)'''
    secondary = self._map[k] # map raise KeyError
    v = secondary.pop()
    if len(secondary) == 0:
      del self._map[k] # no pairs left
    self._n -= 1
    return (k, v)
  
  def find(self, k):
    '''Return arbitrary (k,v) pair with given key (or raise KeyError)'''
    secondary = self._map[k] # may raise KeyError
    return (k, secondary[0])
  
  def find_all(self, k):
    '''Generate iteration of all (k,v) pairs with given key'''
    secondary = self._map.get(k, []) # empty list, by default
    for v in secondary:
      yield (k, v)